**prec_quadrature**: Python implementation of the MATLAB quadrature-finding algorithm
contained in this directory, so implemented to use high-precision floating point
representations

The core idea is that overall convergence of the MATLAB algorithm might be limited
by basic roundoff error: the problem $\sum_k w(k) M_j(x_k) = F_j$ has a near-singular Jacobian
in the vicinity of known-good solutions, so even small errors in the calculation of
the RHS can lead to a stagnated, unconverged iteration.

In [ ]:
import mpmath # Arbitrary-precision floating point
from mpmath import mpf # Object for an arbitrary-precision float
from mpmath import mp  # Control structure for changing precision
import matplotlib # Visualization
import matplotlib.pyplot as plt

# Disable any multithreading inside numpy/MKL
import os
os.environ['MKL_NUM_THREADS'] = '1'


import numpy as np # Arrays for low (machine)-precision floating point values
import scipy.spatial
import quadfuncs # Functions to generate quadrature moments
import calcquad # Functions to evaluate moments and generate the quadrature rule
% matplotlib inline

## One-dimensional and Cartesian Product quadrature

The core algorithm of the quadrature-finding algorithm is to find a small set of
node locations ($\vec{x}_i$) and weights ($w_i$) where 
$\sum_i w_i M_j(\vec{x}_i) = \int_\Omega M_j(\vec{x}) dA$ for the moment functions $M_j(\vec{x})$.
Computing this requires a sufficiently-accurate right-hand side for each element,
and in turn this is most easily given from an impractically large quadrature rule,
generated by a Cartesian product.

## Moments and orthogonalization

Spherical harmonics can be expressed in terms of Cartesian coordinates on the sphere:
the zeroth-order harmonic is the constant 1, the first-order harmonics are $(x,y,z)$,
the second-order harmonics are $(x^2,xy,y^2,xz,yz)$, and so on. These form the natural 
basis for a quadrature rule on the sphere, in the same way that polynomials form the 
natural basis for quadrature on the real line.

This has two complications, however:

1. There's a difference between a quadrature rule where "spherical harmonics up to order N
are integrated exactly" and one where "the error is proportional to $\Delta x^N$.''
2. The spherical harmonics are orthogonal over the entire sphere, but they are most emphatically
not orthogonal on any particular element.

These complications are related.  Consider an element centered at one of the poles.  There,
local variation is described extremely well -- to the proper error estimate -- using only
spherical harmonics involving powers $x$ and $y$, so a quadrature rule that integrates only
those terms exactly has the right properties.  However, such a quadrature rule will not exactly
integrate $z$.

In this element, $z = \sqrt{1-x^2-y^2} \approx 1-(x^2+y^2)/2$, and it is nearly collinear with
the lower-degree spherical harmonics of $x$ and $y$ only.  This causes a serious numerical issue
as elements become smaller, and we must resort to orthogonalization and high-precision arithmetic.

We accomplish this using a semi-analytical framework.  The ancillary module `quadfuncs` uses SymPy to compute a set of approximation functions $\phi_n = x^i y^j z - P_n(x,y)$ for a polynomial $P$, such that $\phi_n$ is orthogonal (over $|x|, |y| \leq \epsilon$) to the full set of moment functions that do not contain a 'z' term _and_ $\phi_k$ for $k < n$.  This process requires the use of high precision for intermediate results, but the result is a set of functions that can calculate $\phi$ using ordinary doubles.

In [ ]:
# Set parameters 
NTri = 6
NSph = 6
r0 = 1e-3

print('Computing moment functions for order %d x %d' % (NTri,NSph))
(funcs_sph, z_poly, z_approx) = quadfuncs.get_sph_funcs(NSph,NTri,r0,debug_funcs=True)
funcs_tri = quadfuncs.get_tri_funcs(NTri,r0)
print('... complete')

In [ ]:
funcs = funcs_tri + funcs_sph

In [ ]:
x1d = r0*np.linspace(-1,1,64)
y1d = r0*np.linspace(-1,1,64)
(x2d,y2d) = np.meshgrid(x1d,y1d,indexing='xy')
(moms) = calcquad.eval_moments(x2d,y2d,funcs)

In [ ]:
NBoxX = np.int(np.ceil(np.sqrt(moms.shape[0])))
NBoxY = np.int(np.ceil(moms.shape[0]*1.0/NBoxX))
for idx in range(moms.shape[0]):
    plt.subplot(NBoxX,NBoxY,idx+1)
    plt.pcolormesh(x1d/r0,y1d/r0,moms[idx,:,:].T,
                  vmin=-2,vmax=2,
                   cmap='RdYlBu_r')
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
plt.suptitle('Moment funtions');

## Hexagonal quadrature

The above-defined moment functions are now well-conditioned on grids with a length scale of $r_0$.  They're not exactly orthogonal, but the Netwon's Method procedure is capable of dealing with mild amounts of linear dependence.

The quadrature-generation algorithm involves taking a candidate set of points and weights and changing it (iteratively) to match the integrals of the moment functions, which are in turn generated by a high-order, tensor-product quadrature rule involving many points.

Functions to accomplish this are included in the creatively-named `calcquad` module.

### Exact quadrature

We begin by exploring just what kind of tensor-product quadrature is necessary for sufficient (near numerical-precision) accuray.

In [ ]:
mp.dps = 50
r0 = mpf('.001')
# Create an irregular hexagon, using a LCG to generate "random" but deterministic variation
xhex = np.array([r0*(mp.cos((2*mp.pi*i)/6) + 0.1*(2*((mp.sqrt(2)*i) % 1) - 1)) \
                     for i in range(6)])
yhex = np.array([r0*(mp.sin((2*mp.pi*i)/6) + 0.1*(2*((mp.sqrt(3)*i) % 1) - 1)) \
                     for i in range(6)])

# Very high-order quadrature rule
(q1x, q1w) = calcquad.gauss(N=50,dps=50)
(q2x, q2y, q2w) = calcquad.hex_quad(xhex,yhex,q1x,q1w)
q2w = q2w*calcquad.gnom_weight(q2x,q2y)

# Evaluate moment functions at these points to get ''exact'' answer
moms = calcquad.eval_moments(q2x,q2y,funcs)
exact_quad = np.dot(moms,q2w)

# Perform a binary search to find the smallest tensor-product quadrature necessary for the
# prescribed tolerance
low_N = 1
high_N = 50
try_N = 10
current_err = np.inf

while (high_N > low_N):
    (q1x, q1w) = calcquad.gauss(N=try_N,dps=50)
    (q2x, q2y, q2w) = calcquad.hex_quad(xhex,yhex,q1x,q1w)
    q2w = q2w*calcquad.gnom_weight(q2x,q2y)
    moms = calcquad.eval_moments(q2x,q2y,funcs)
    
    trial_quad = np.dot(moms,q2w)
    current_err = np.max(np.abs(exact_quad-trial_quad)/exact_quad[0])
    print('%2d: %e' % (try_N,current_err))
    if (current_err > 1e-16): # Guess is too low
        low_N = try_N + 1 # Lowest possible value is more than the current test
        try_N = (low_N + high_N) // 2
    else:
        high_N = try_N
        try_N = (low_N + high_N) // 2
print('Found acceptable quadrature at %d points' % high_N)

In [ ]:
# Create a 1D quadrature rule of this size
(q1x,q1w) = calcquad.gauss(N=10,dps=20)
q1x = q1x.astype('double')
q1w = q1w.astype('double')

### Quadrature point rules

Next, we want guidelines on how many points are necessary for Nth order quadrature (both with and without exact spherical harmonics) on irregular hexagons.  The absolute lower bound here would be (# functions)/3, since each quadrature point has three degrees of freedom (x,y,weight), but this lower bound is not tight.

This test proceeds in two phases.  First, we construct a quadrature rule for the regular spherical hexagon by iterative reduction, beginning from a larger number of candidate quadrature points.  If that process reaches the target (and passes a santity check where quadrature points are not too far outside the hexagon), then we proceed to calculate quadrature rules for 100 randomly-perturbed hexagons.  In some cases, a rule that works for the regular hexagon breaks for perturbed hexagons, perhaps because of a lack of symmetry.

In [ ]:
r0 = 1e-3
quadrules = {} # Dictionary of parameters for acceptable quadrature rules
for order in range(6,0,-1):
    NTri = order
    for NSph in [NTri,0]:
        print('Testing quadrature order %d x %d' % (NTri,NSph))
        funcs_sph = quadfuncs.get_sph_funcs(NSph,NTri,r0,prec=75)
        funcs_tri = quadfuncs.get_tri_funcs(NTri,r0)
        funcs = funcs_tri + funcs_sph
        
        NMin = int(np.ceil(len(funcs)/3.0))
        good_rule = False
        
        for N_targ in [NMin,NMin+1,NMin+2,NMin+3]:
            for N_init in [#N_targ,
                           N_targ+5,
                           len(funcs_sph)+len(funcs_tri)]:
                print('   Start (%d), target (%d)' % (N_init, N_targ))
                (xreg,yreg,wreg,loop_err,itercount,quad_reg) = calcquad.calc_quad_init(N_init,N_targ,funcs,
                                                                                       q1x,q1w,r0)
                if (loop_err > 1e-19):
                    print('   ... failed (error)')
                    continue
                if (any(xreg**2+yreg**2 > (1.2*r0)**2)):
                    print('   ... failed (sanity)')
                    continue
                else:
                    print('   ... success (%d iter)' % itercount)
                # Test adjustment
                failed = 0
                adjust = 0
                initial = 0

                iters = 0
                for it in range(100):
                    xhex = r0*(np.cos(2*np.pi*np.arange(6)/6) + 1e-1*(2*np.random.rand(6) - 1))
                    yhex = r0*(np.sin(2*np.pi*np.arange(6)/6) + 1e-1*(2*np.random.rand(6) - 1))

                    (xlow,ylow,wlow,loop_err,itercount) = calcquad.calc_hex_quad(xhex,yhex,xreg,yreg,wreg,
                                                                                 quad_reg,funcs,q1x,q1w,r0)
                    if (loop_err < 1e-14*quad_reg[0]*N_targ**0.5):
                        adjust += 1
                        iters += itercount
                        continue
                    loop_err_adj = loop_err/quad_reg[0]
                    # In testing, re-initializing the rule for an element seems to never work when ordinary
                    # adjustment fails.
#                     (xlow,ylow,wlow,loop_err,itercount,_) = calcquad.calc_quad_init(N_init,N_targ,
#                                                                                     funcs,q1x,q1w,r0,
#                                                                                     xhex,yhex)
#                     if (loop_err < 1e-14*quad_reg[0]):
#                         initial += 1
#                         iters += itercount
#                         continue
                    loop_err_init = loop_err/quad_reg[0]
                    failed += 1
                    break
                if (failed > 0):
                    print('   ... adjustment failed (%d, %e)' % (it, loop_err_adj))
                    continue
                else:
                    print('   ... adjustment success (%d, %d), %d iter' % (adjust, initial,iters))
                    good_rule = True
                    quadrules[(NTri,NSph)] = (N_init,N_targ)
                    
#                     plt.figure()
#                     plt.plot(np.cos(2*np.pi*np.arange(7)/6),np.sin(2*np.pi*np.arange(7)/6),'k-')
#                     plt.plot(np.cos(2*np.pi*np.arange(65)/64),np.sin(2*np.pi*np.arange(65)/64),'b-')
#                     plt.plot(xreg/r0,yreg/r0,'r.')
#                     plt.show()
                                 
                    break
            if (good_rule): 
                # Plot moment functions
                plt.figure(figsize=(3.5,2.75))
                x1d = r0*np.linspace(-1,1,64)
                y1d = r0*np.linspace(-1,1,64)
                (x2d,y2d) = np.meshgrid(x1d,y1d,indexing='xy')
                (moms) = calcquad.eval_moments(x2d,y2d,funcs)
                NBoxY = np.int(np.ceil(np.sqrt(moms.shape[0])))
                NBoxX = np.int(np.ceil(moms.shape[0]*1.0/NBoxY))
                for idx in range(moms.shape[0]):
                    ax = plt.subplot(NBoxX,NBoxY,idx+1)
                    plt.contourf(x1d/r0,y1d/r0,np.maximum(-2,np.minimum(2,moms[idx,:,:].T)),
                                 np.linspace(-2,2,32),
                                   cmap='RdYlBu_r')
                    plt.gca().set_xticks([])
                    plt.gca().set_yticks([])
                plt.suptitle('Moment funtions to order %d' % (NTri),fontsize=10);
#                 plt.tight_layout()
                plt.savefig('figs/moments_%d_%d.pdf' % (NTri,NSph),dpi=150)
                plt.show()
                break
        

In [ ]:
quadrules

## Import and process a grid file

This section of the notebook imports and processes a text-based grid file.  This grid file opens with a single line containing an integer, then &lt;integer&gt; lines each containing three floating-point values that define a point on the unit sphere.

These points are generator locations for the primal grid, which consists of spherical polygons (hexagons and pentagons, typically).  The convex hull of these points defines a dual grid of spherical triangles.

In [ ]:
def build_dual_centers(chull,primal_generators):
    '''build_dual_centers: calculate the centers of a dual grid (triangles) given primal generator 
    locations and a convex hull'''

    # Build barycenters and voronoi centers of the dual triangles

    dual_bcent = np.zeros((len(chull.simplices),3),dtype=np.double)
    dual_vcent = np.zeros((len(chull.simplices),3),dtype=np.double)

    for (idx, tri) in enumerate(dual_faces):
        triverts = primal_generators[tri,:]

        # The barycenter is the average of the triangle vertices, projected to the sphere
        bcent = np.mean(triverts,axis=0)
        bcent = bcent/np.sum(bcent**2)**0.5
        dual_bcent[idx,:] = bcent

        # The voronoi center is the point orthogonal to the triangle edges; see Muira [2005], equation 2
        vcent = np.cross(triverts[1,:]-triverts[0,:],triverts[2,:]-triverts[0,:])
        vcent = vcent/np.sum(vcent**2)**0.5 * np.sign(np.dot(vcent,triverts[0,:]))
        dual_vcent[idx,:] = vcent
        
    return(dual_bcent,dual_vcent)

In [ ]:
# Build list of vertices (by face) for the primal grid
def build_verts(primal_generators,dual_faces,dual_centers):
    '''build_verts: Build a list of vertices (triangle centers) for the primal grid (hexagon/pentagon)'''
    primal_vertices = [ [] for x in range(len(primal_generators))]

    # Iterate through dual grid vertices, associating each face with its vertex (primal face)
    for (idx, tri) in enumerate(dual_faces):
        for t in tri:
            primal_vertices[t].append(idx)

    # Now, go through this list and order the vertices in CCW direction
    for (idx, verts) in enumerate(primal_vertices):
        vert_coords = dual_centers[verts,:]

        # Compute an approximate center of the polygon
        poly_center = np.mean(vert_coords,axis=0)

        # Generate false east/north coordinates, to determine 
        false_east = vert_coords[0,:] - poly_center
        false_north = np.cross(poly_center,false_east)

        false_x = np.dot(false_east,vert_coords.T)
        false_y = np.dot(false_north,vert_coords.T)

        angle = np.mod(np.arctan2(false_y,false_x)+2*np.pi,2*np.pi)
        angle[0] = 0.0

        sorted_idx = np.argsort(angle)
        primal_vertices[idx] = [verts[i] for i in sorted_idx]
    return primal_vertices
    


We can now enumerate the edges of this grid.  Each primal edge (edge of a spherical polygon) has a dual edge (edge of a spherical triangle), so it is defined uniquely by the 4-tuple (primal face 1, primal face 2, dual face 1, dual face 2).  From the data structure given by the ConvexHull, we can build this tuple in two passes:

In [ ]:
def build_edges(dual_faces):
    '''build_edges: Build a list of edges (face/vertex connections)'''
    edgedict = {} # Dictionary indexed by (primal face 1, primal face 2)
    def addvert(edge, pf1, pf2, df1):
        # For the given dual edge connecting (pf1) and (pf2), add 
        # the connection (df1) to the edge dictionary

        val = edge.get((pf1,pf2),[])
        val.append(df1)
        edge[(pf1,pf2)]=sorted(val)

    # Pass 1 -- build edges by accumulating primal face / dual face associations
    for (idx,tri) in enumerate(dual_faces):
        stri = sorted(tri) # Sort vertices of the triangle to give a canonical order
        addvert(edgedict,stri[0],stri[1],idx)
        addvert(edgedict,stri[1],stri[2],idx)
        addvert(edgedict,stri[0],stri[2],idx)

    # Pass 2 -- build an edge array, where each entry contains:
    # (pf1, pf2, pv1, pv2)
    # to represent the edge between faces pf1/pf2 that consists of vertices pv1/pv2
    edges = np.zeros((len(edgedict),4),dtype=np.int)
    for (idx,(k, v)) in enumerate(edgedict.iteritems()):
        if (v[0] < v[1]):
            edges[idx,:] = [k[0], k[1], v[0], v[1]]
        else:
            edges[idx,:] = [k[0], k[1], v[1], v[0]]

    # Sort the edges array so that faces are in lexical order
    edges = edges[edges[:,1].argsort()]
    edges = edges[edges[:,0].argsort()]
    return edges

Define test functions to evaluate the accuracy of the resulting quadrature rules.  `test_sph0` through `test_sph3` are polynomials in Cartesian coordinates, which are also linear combinations of spherical harmonics.  Quadrature rules which include the "spherical" moments (of sufficiently high order) should integrate these functions exactly, up to the prescribed tolerance.

In [ ]:
def test_tanh(xyz):
    '''test_tanh: tanh test function (Reeger and Fornberg 2015, eqn f_1)'''
    import numpy as np
    return (1+np.tanh(9*(xyz[:,2]-xyz[:,0]-xyz[:,1])))

def test_sph0(xyz):
    '''test_sph0 -- cell area'''
    return(xyz[:,0]**0)

def test_sph1(xyz):
    '''test_sph1: first-order spherical harmonics'''
    return (xyz[:,0]+xyz[:,2])

def test_sph2(xyz):
    '''test_sph2: second-order spherical harmonics'''
    return (xyz[:,0]**2 + xyz[:,0]*xyz[:,2])

def test_sph3(xyz):
    '''test_sph3: third-order spherical harmonics'''
    return xyz[:,0]**3 + xyz[:,1]**3 + xyz[:,0]*xyz[:,1]*xyz[:,2]

test_funcs = ['1','$x+z$','$x^2 + xz$','$x^3 + y^3 + xyz$',
    '$1+\\tanh(9(z-x-y))$']

We need a sufficiently accurate quadrature rule to generate reference values for the test integrals, so copy the tensor-product generation code from `calcquad` and apply the rule directly to provided functions.  There is no need to save the resulting quadrature rule, and in fact doing so would chew through memory.

In [ ]:
def control_quad(verts,q1x,q1w,funcs):
    '''control_quad: calculate tensor-product quadrature of provided functions over a single cell'''
    
    import numpy as np
    import calcquad
    
    # Calculate an approximate center of the cell, by averaging the vertices
    center = np.mean(verts,axis=0)
    # Project back to the unit sphere
    center /= np.linalg.norm(center)

    if (verts.shape[0] == 5):
        # We have a hexagon, not a pentagon, so duplicate a vertex
        verts = verts[[0]+range(5),:]

    t_vec = center # t is the outward coordinate, which takes the place of 'z'
    # Pick one of the vertex directions as 'r' (x-substitute)
    r_vec = verts[0,:] - center
    r_vec -= t_vec*np.dot(r_vec,t_vec)
    r_vec /= np.linalg.norm(r_vec)

    # Pick the other direction as 's' (y-substitute)
    s_vec = np.cross(t_vec,r_vec)

    # These vectors give a transform matrix
    tmat = np.array([r_vec,s_vec,t_vec]) # Convert x/y/z to r/s/t
    tmati = np.linalg.inv(tmat) # Convert r/s/t to x/y/z

    # Convert the hexagon vertices to {r,s,t} space
    proj_verts = np.dot(tmat,verts.T)

    # Divide the coordinates by their third component to effect the gnomonic projection
    hex_gr = proj_verts[0,:]/proj_verts[2,:]
    hex_gs = proj_verts[1,:]/proj_verts[2,:]

    (quad_gr,quad_gs,quad_w) = calcquad.hex_quad(hex_gr,hex_gs,q1x,q1w)
    quad_w *= calcquad.gnom_weight(quad_gr,quad_gs)
    
    # Convert the quadrature points on the gnonomic (r,s) plane back to a rotated sphere:

    # First, place the points on the tangent plane
    quad_verts = np.dot(tmat.T,np.vstack((quad_gr, quad_gs, quad_gr**0)))

    # Then, normalize to the unit sphere
    quad_verts = quad_verts / np.sqrt(np.sum(quad_verts**2,axis=0))
    quad_verts = quad_verts.T

    
    out = map(lambda f: np.dot(quad_w,f(quad_verts)), funcs)
    return out


In [ ]:
def plot_ortho(verts,points,center,east):
    '''Helper function to plot a polygon and points in an orthographic projection'''
    
    # Vectors tangent to the sphere
    t_vec = center
    t_vec /= np.linalg.norm(t_vec)
    r_vec = east - np.dot(east,t_vec)*t_vec
    r_vec /= np.linalg.norm(r_vec)
    s_vec = np.cross(t_vec,r_vec)
    tmat = np.array([r_vec,s_vec,t_vec])
    
    for idx in range(-1,verts.shape[0]-1):
        v1 = verts[idx,:]
        v2 = verts[idx+1,:]
        
        gnom_1 = np.dot(tmat,v1)
        gnom_1 /= gnom_1[2]
        gnom_2 = np.dot(tmat,v2)
        gnom_2 /= gnom_2[2]
        
        ignom = gnom_1[:,np.newaxis] + (gnom_2-gnom_1)[:,np.newaxis]*np.linspace(0,1,32)
        iortho = ignom / (1+ignom[0,:]**2 + ignom[1,:]**2)**0.5
        
        plt.plot(iortho[0,:],iortho[1,:],'k-')
        
    proj_pts = np.dot(tmat,points.T)
    plt.plot(proj_pts[0,:],proj_pts[1,:],'r.')
        

In [ ]:
# Scaffolding for parallel generation of reference results
import ipyparallel
try:
    rc.queue_status()
except (NameError, IOError):
    rc = ipyparallel.Client()
    view = rc[:]

In [ ]:
# Loop over all the admissible quadrature rules and over a selection of grid refinement 
# levels to produce quadrature rules and error estimates
qtensor_dict = {}
qcalc_dict = {}
verts_dict = {}
for (NTri,NSph) in sorted(quadrules.keys()):
    quad_tensor = []
    quad_calc = []
    qp_arr = []
    qw_arr = []
    (N_start, N_targ) = quadrules[(NTri,NSph)]
    for LEVEL in range(1,7):
        gridfile = '../iModel/grid/icos_pol_scvt_h1_%d.xyz' % LEVEL

        with open(gridfile,'r') as genfile:
            npoints = int(genfile.readline())
            primal_generators = np.array([[float(f) for f in l.split()] for l in genfile])

        chull = scipy.spatial.ConvexHull(primal_generators)
        dual_faces = chull.simplices # Triangles, indexed by generator
        (dual_bcent,dual_vcent) = build_dual_centers(chull,primal_generators)
        primal_vertices = build_verts(primal_generators,dual_faces,dual_vcent)
        edges = build_edges(dual_faces)

        (quad_points, quad_weights, rel_errs, total_iter,
         xreg, yreg, wreg) = calcquad.quadgrid(primal_vertices,dual_vcent,q1x,q1w,NTri,NSph,N_start,N_targ)

        qp_arr.append(np.array(quad_points))
        qw_arr.append(np.array(quad_weights))

        # Get control quadratures
        verts = [dual_vcent[primal_vertices[i],:] for i in range(len(primal_vertices))]
        if LEVEL not in verts_dict:
            verts_dict[LEVEL] = verts
        view.push(dict(control_quad = control_quad,
                       test_tanh=test_tanh,
                       test_sph0 = test_sph0,
                       test_sph1 = test_sph1,
                       test_sph2 = test_sph2,
                       test_sph3 = test_sph3,
                       q1x=q1x,q1w=q1w))
        quad_tensor.append(np.array(view.map_sync(lambda v: 
                                                  control_quad(v,q1x,q1w,
                                                               [test_sph0,test_sph1,test_sph2,test_sph3,test_tanh]), 
                                                  verts)))

        qp2d = np.array(quad_points).reshape((-1,3))
        qw2d = np.array(quad_weights)

        quadcalc_arr = np.array(
            map(lambda f: np.sum(f(qp2d).reshape((-1,N_targ))*qw2d,axis=1),
                [test_sph0,test_sph1,test_sph2,test_sph3,test_tanh])).T
        quad_calc.append(quadcalc_arr)

        if (LEVEL == 2):
            # Plot initial and final quadrature rules at a sample hexagon
            r0 = (4.0/len(primal_vertices))**0.5 
            xhex_g = r0*np.cos(np.arange(6)*2*np.pi/6) # Regular hexagon
            zhex_g = r0*np.sin(np.arange(6)*2*np.pi/6)
            yhex = (1+xhex_g**2 + zhex_g**2)**-0.5
            vhex = np.array([xhex_g*yhex,yhex,zhex_g*yhex]).T
            # Regular quadrature points, projected to sphere
            zreg = (1+xreg**2+yreg**2)**-0.5
            ax = plt.figure(figsize=(3.5,2.75))
            plot_ortho(vhex,np.array((xreg*zreg,zreg,yreg*zreg)).T,np.array([0.0,1,0]),np.array([1.0,0,0]))
            plt.title('Regular hexagon (%d, %d)' % (NTri,NSph))
            plt.savefig('figs/regquad_%d_%d.pdf' % (NTri,NSph), dpi=150)
            plt.show()
            IDX = 12
            ax = plt.figure(figsize=(3.5,2.75))
            cent = np.mean(verts[IDX],axis=0)
            plot_ortho(verts[IDX],quad_points[IDX],cent,verts[IDX][0,:] - cent)
            plt.title('Sample cell (%d, %d)' % (NTri,NSph))
            plt.savefig('figs/sampquad_%d_%d.pdf' % (NTri,NSph), dpi=150)
            plt.show()


        # Write out 0-indexed grid information

        if (False):
            # Write vertex data: each entry is the (x,y,z) coordinate of a primal-grid vertex
            with open('vertices_%d.xyz' % LEVEL,'w') as vertfile:
                # One header line: number of vertices
                vertfile.write('%d\n' % dual_vcent.shape[0])
                for idx in range(dual_vcent.shape[0]):
                    vertfile.write(' '.join(['%+.17e' % dual_vcent[idx,j] for j in range(3)]) + '\n')


            # Write edge data: each entry is the (v1, v2, f1, f2) tuple (zero-indexed) of
            # faces and vertices corresponding to a grid edge
            with open('edges_%d.ij' % LEVEL,'w') as edgefile:
                edgefile.write('%d\n' % edges.shape[0])
                for idx in range(edges.shape[0]):
                    edgefile.write('%d %d %d %d\n' % tuple(edges[idx,:]))

            # Write face data: each entry is the (v0..v[45]) tuple (zero-indexed) of vertices
            # corresponding to a grid face, in anticlockwise order
            with open('faces_%d.ij' % LEVEL,'w') as facefile:
                facefile.write('%d\n' % len(primal_vertices))
                for vlist in primal_vertices:
                    facefile.write(' '.join('%d' % v for v in vlist) + '\n')

            with open('quad_%d_%dx%d.xyzw' % (LEVEL,NTri,NSph),'w') as qfile:
                qfile.write('%d %d\n' % (len(quad_weights), len(quad_weights[0])))
                for (idx) in range(len(quad_points)):
                    for (qp,qw) in zip(quad_points[idx],quad_weights[idx]):
                        qfile.write('%+.17e %+.17e %+.17e %+.17e\n' % (tuple(qp)+(qw,)))
                        
    qtensor_dict[(NTri,NSph)] = quad_tensor
    qcalc_dict[(NTri,NSph)] = quad_calc

In [ ]:
# Plot error convergence curves

lines = []
labels = []
plt.figure(figsize=(3.5,2.75))
for NTri in range(1,7):
    NSph = 0
    quad_calc = qcalc_dict[(NTri,NSph)]
    quad_tensor = qtensor_dict[(NTri,NSph)]
    (N_start,N_targ) = quadrules[(NTri,NSph)]
    diffs = []
    for idx in range(len(quad_calc)):
        quad_diff = (quad_calc[idx] - quad_tensor[idx]).T/quad_calc[idx][:,0]
        diffs.append(np.max(np.abs(quad_diff),axis=1))
#         print(diffs[-1])
    diffs = np.array(diffs)
    pts = np.array([g.shape[0] for g in quad_calc],dtype=np.double)*N_targ
#     plt.loglog(pts,diffs[:,1],linestyle=':',marker='.',color=plt.cm.tab10(NTri-1))
#     plt.loglog(pts,diffs[:,2],linestyle='--',marker='.',color=plt.cm.tab10(NTri-1))
#     plt.loglog(pts,diffs[:,3],linestyle='-.',marker='.',color=plt.cm.tab10(NTri-1))
    ln, = plt.loglog(pts,diffs[:,4],linestyle='-',marker='.',color=plt.cm.tab10(NTri-1))
    lines.append(ln)
    labels.append('Order %d' % (NTri))

    
plt.xlabel('# Points')
plt.ylabel('$||\cdot||_\infty$ error')
plt.legend(lines,labels)
plt.title('Quadrature errors (triangular)')
plt.axis((42,800000,3e-10,1e0))
plt.tight_layout()
plt.savefig('figs/tri_err.pdf',dpi=150)
plt.show()

lines = []
labels = []
plt.figure(figsize=(3.5,2.75))
for NTri in range(1,6):
    NSph = NTri
    quad_calc = qcalc_dict[(NTri,NSph)]
    quad_tensor = qtensor_dict[(NTri,NSph)]
    (N_start,N_targ) = quadrules[(NTri,NSph)]
    diffs = []
    for idx in range(len(quad_calc)):
        quad_diff = (quad_calc[idx] - quad_tensor[idx]).T/quad_calc[idx][:,0]
        diffs.append(np.max(np.abs(quad_diff),axis=1))
#         print(diffs[-1])
    diffs = np.array(diffs)
    pts = np.array([g.shape[0] for g in quad_calc],dtype=np.double)*N_targ
#     plt.loglog(pts,diffs[:,1],linestyle=':',marker='.',color=plt.cm.tab10(NTri-1))
#     plt.loglog(pts,diffs[:,2],linestyle='--',marker='.',color=plt.cm.tab10(NTri-1))
#     plt.loglog(pts,diffs[:,3],linestyle='-.',marker='.',color=plt.cm.tab10(NTri-1))
    ln, = plt.loglog(pts,diffs[:,4],linestyle='-',marker='.',color=plt.cm.tab10(NTri-1))
    lines.append(ln)
    labels.append('Order %d' % (NTri))

    
plt.xlabel('# Points')
plt.ylabel('$||\cdot||_\infty$ error')
plt.legend(lines,labels)
plt.title('Quadrature errors (complete)')
plt.axis((42,800000,3e-10,1e0))
plt.tight_layout()
plt.savefig('figs/full_err.pdf',dpi=150)
plt.show()

In [ ]:
# Plot a sample visualization of quadrature error.  For the pre-selected values and the tanh
# test function, the quadrature error should be concentrated in the small transition band

polys = []
colors = []
pts = []
NTri = 4; NSph = 4
SIZE_IDX = -1
quad_calc = qcalc_dict[(NTri,NSph)]
quad_tensor = qtensor_dict[(NTri,NSph)]
quad_diff = ((quad_calc[SIZE_IDX] - quad_tensor[SIZE_IDX]).T/quad_calc[SIZE_IDX][:,0]).T
cmap = plt.get_cmap('RdYlBu')
TESTFUNC=4

# Plot true function
vmax = np.max(np.abs(quad_calc[SIZE_IDX][:,TESTFUNC]/quad_calc[SIZE_IDX][:,0]))
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
for (idx,v) in enumerate(verts):
    if (np.any(v[:,1] < 0)): continue
    polys.append(Polygon(v[:,[0,2]],closed=True))
    colors.append(quad_calc[SIZE_IDX][idx,TESTFUNC]/quad_calc[SIZE_IDX][idx,0])
fig, ax = plt.subplots()
p = PatchCollection(polys, cmap=plt.get_cmap('RdYlBu_r'), alpha=1.0)
p.set_array(np.array(colors))
p.set_clim([-vmax,vmax])
p.set_edgecolor(None)
plt.colorbar(p)
ax.add_collection(p)
ax.autoscale()
ax.set_title(test_funcs[TESTFUNC])

# pos_pts = (qp2d[:,1] > 0)
# plt.plot(qp2d[pos_pts,0],qp2d[pos_pts,2],'k.',markersize=4)
plt.show()

# Plot quadrature difference
vmax = np.max(np.abs(quad_diff[:,TESTFUNC]))
cmap = plt.get_cmap('RdYlBu')
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
for (idx,v) in enumerate(verts):
    if (np.any(v[:,1] < 0)): continue
    polys.append(Polygon(v[:,[0,2]],closed=True))
    colors.append(quad_diff[idx,TESTFUNC])
    pts.append(quad_points[idx])
fig, ax = plt.subplots()
p = PatchCollection(polys, cmap=plt.get_cmap('RdYlBu_r'), alpha=1.0)
p.set_array(np.array(colors))
p.set_clim([-vmax,vmax])
p.set_edgecolor(None)
plt.colorbar(p)
ax.add_collection(p)
ax.autoscale()
ax.set_title('%s error' % test_funcs[TESTFUNC])
pts = np.array(pts).reshape((-1,3))
# plt.plot(pts[:,0],pts[:,2],'k.',markersize=2)
plt.show()